In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
K = keras.backend
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import pickle

In [2]:
# Load comparative models


import mppca
from nn_mppca import NNMPPCA
from vae import VAE
from donut import DONUT
from dagmm import DAGMM
from damppca import DAMPPCA
from vaemppca import VAEMPPCA
from vaemppca1 import VAEMPPCA1
from vaemppca2 import VAEMPPCA2
from vaemppca3 import VAEMPPCA3


# Experiement 1

In [3]:


def load_data(path, filename):
    csv_load_path = os.path.join(path, filename)
    return pd.read_csv(csv_load_path, index_col=0)

def save_data(df, path, filename):
    csv_save_path = os.path.join(path, filename)
    df.to_csv(csv_save_path)
    
def load_file(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [81]:
# Load data
data_path = 'datasets/spring/' # path to the dataset
data = load_data(data_path, 'spring_data.csv')

# Remove control features
a = np.array([0, 1]).astype('float64')
b = np.array([1]).astype('float64')
c = np.array([0]).astype('float64')
columns = data.columns
for col in columns.to_list():
    if np.array_equal(data[col].unique(), a) or np.array_equal(data[col].unique(), b) or np.array_equal(data[col].unique(), c):
        columns = columns.drop([col])
        
data = data[columns]
data


,ECONCTRL,HWC-VLV,CHWC-VLV,EA-DMPR,RA-DMPR,OA-DMPR,SF-WAT,RF-WAT,HWP-GPM,CHWP-GPM,...,CHWC_GPM,E_hcoil,E_ccoil,E_SF,E_RF,E_ZONE_I,E_ZONE_W,E_ZONE_S,E_ZONE_E,label
0,3,100,0.000,0.0,0.0,0.0,35.129,35.129,0.448,12.991,...,-0.253,0.0,0.000,0.035,0.035,0.004,0.006,0.004,0.005,0
1,3,100,0.000,0.0,0.0,0.0,35.129,33.958,0.438,13.035,...,-0.186,0.0,0.000,0.035,0.034,0.004,0.005,0.005,0.005,0
2,3,100,0.000,0.0,0.0,0.0,35.129,35.129,0.438,13.049,...,-0.185,0.0,0.000,0.035,0.035,0.004,0.006,0.005,0.005,0
3,3,100,0.000,0.0,0.0,0.0,35.129,33.958,0.427,12.962,...,-0.185,0.0,0.000,0.035,0.034,0.004,0.005,0.005,0.005,0
4,3,100,0.000,0.0,0.0,0.0,33.958,35.129,0.406,13.035,...,-0.187,0.0,0.000,0.034,0.035,0.004,0.006,0.005,0.005,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20155,3,100,23.347,40.0,40.0,40.0,514.202,217.862,0.011,13.339,...,2.435,0.0,5.888,0.514,0.218,0.837,0.180,0.000,0.081,7
20156,3,100,22.924,40.0,40.0,40.0,495.461,217.862,0.011,13.398,...,2.466,0.0,6.050,0.495,0.218,0.827,0.176,0.001,0.080,7
20157,3,100,22.090,40.0,40.0,40.0,503.660,219.034,0.011,13.368,...,2.480,0.0,6.038,0.504,0.219,0.839,0.389,0.000,0.079,7
20158,3,100,21.389,40.0,40.0,40.0,517.716,219.034,0.011,13.398,...,2.369,0.0,5.779,0.518,0.219,0.745,0.662,0.001,0.879,7


In [107]:
# Inject missing values
X, y = data.iloc[:, :-1], data.iloc[:, -1]
X.replace(to_replace=0, value=1e-4, inplace=True)

# Convert conventional target to binary targe
y_bi = np.where(y==0, 0, 1)
y_bi

injection_matrix = np.random.choice([0, 1], size=X.shape, p=[0.5, 0.5])
X_injected = np.multiply(injection_matrix, X)

# Results

In [ ]:
results = []

## 1. GMM 

In [44]:
from sklearn.mixture import GaussianMixture
precs = []
recals = []
fscores = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    gm_clf = GaussianMixture(n_components=3, random_state=0).fit(X_train)
    y_pred = gm_clf.score_samples(X_test)
    anomaly_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred <= anomaly_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)
    
print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'GMM', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


Energy thleshold to detect anomaly : -369893378.411
Energy thleshold to detect anomaly : -370367688.678
Energy thleshold to detect anomaly : -370362467.946
Energy thleshold to detect anomaly : -375754043.800
Energy thleshold to detect anomaly : -375753176.739
 Precision = 1.000 ± 0.000
 Recall    = 0.996 ± 0.002
 F1-Score  = 0.998 ± 0.001
 F1-scores:
           0
0  0.998811
1  0.998316
2  0.998712
3  0.996343
4  0.996934


## 2. MPPCA

In [8]:
def mppca_predict(X, phi, mu, W, sigma2):
    d = X.shape[-1]
    C = sigma2[:, np.newaxis, np.newaxis]*np.eye(d) + np.einsum('ikl,ilm->ikm', W, np.transpose(W, [0, 2, 1]))
    min_vals = np.diag(np.ones(d, dtype=np.float32)) * 1e-3
    L = np.linalg.cholesky(C + min_vals[np.newaxis, :, :])
    z_centered_2 = X[:, np.newaxis, :] - mu[np.newaxis, :, :] 
    v = tf.linalg.triangular_solve(L, np.transpose(z_centered_2, [1, 2, 0]))  # kli
    # log(det(cov)) = 2 * sum[log(diag(L))]
    log_det_cov = 2.0 * tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)), axis=1)

    # To calculate energies, use "log-sum-exp"
    logits = tf.math.log(phi[:, np.newaxis]) - 0.5 * (tf.reduce_sum(tf.square(v), axis=1)
                                                      + d * tf.math.log(2.0 * tf.constant(np.pi, dtype='float64')) 
                                                      + log_det_cov[:, np.newaxis])
    res_probs = - tf.reduce_logsumexp(logits, axis=0)
    
    return res_probs.numpy()

p = 3 # We expect 3 clusters
q = 5 # We expect 1-dimensional subspaces
niter = 10
precs = []
recals = []
fscores = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    phi, mu, W, sigma2, clusters = mppca.initialization_kmeans(X_train, p, q)
    phi, mu, W, sigma2, R, L, sigma2hist = mppca.mppca_gem(X_train, phi, mu, W, sigma2, niter)
    y_pred = mppca_predict(X_test, phi, mu, W, sigma2)
    # Energy thleshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)
    
print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'MPPCA', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


..........Energy thleshold to detect anomaly : 168.497
..........Energy thleshold to detect anomaly : 169.259
..........Energy thleshold to detect anomaly : 169.352
..........Energy thleshold to detect anomaly : 169.586
..........Energy thleshold to detect anomaly : 169.026
 Precision = 0.541 ± 0.005
 Recall    = 0.539 ± 0.003
 F1-Score  = 0.540 ± 0.004
 F1-scores:
           0
0  0.547248
1  0.538797
2  0.537429
3  0.540140
4  0.535315


## 3. IF

In [9]:
from sklearn.ensemble import IsolationForest
precs = []
recals = []
fscores = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    forest_clf = IsolationForest(random_state=0).fit(X_train)
    y_pred = forest_clf.score_samples(X_test)
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred <= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)
    
print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'IF', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


Energy thleshold to detect anomaly : -0.537
Energy thleshold to detect anomaly : -0.538
Energy thleshold to detect anomaly : -0.538
Energy thleshold to detect anomaly : -0.537
Energy thleshold to detect anomaly : -0.538
 Precision = 0.486 ± 0.003
 Recall    = 0.485 ± 0.003
 F1-Score  = 0.485 ± 0.001
 F1-scores:
           0
0  0.483984
1  0.484097
2  0.486218
3  0.486374
4  0.485118


## 4. OC

In [10]:
from sklearn.svm import OneClassSVM
precs = []
recals = []
fscores = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
        
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    svm_clf = OneClassSVM(gamma='auto').fit(X_train)
    y_pred = svm_clf.score_samples(X_test)
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred <= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)
    
print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'OC', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


Energy thleshold to detect anomaly : 309.200
Energy thleshold to detect anomaly : 307.658
Energy thleshold to detect anomaly : 302.632
Energy thleshold to detect anomaly : 303.026
Energy thleshold to detect anomaly : 308.818
 Precision = 0.571 ± 0.005
 Recall    = 0.572 ± 0.005
 F1-Score  = 0.571 ± 0.005
 F1-scores:
           0
0  0.579699
1  0.573622
2  0.564734
3  0.571088
4  0.567911


## 5. KDE

In [11]:
from sklearn.neighbors import KernelDensity
precs = []
recals = []
fscores = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
        
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    kde_clf = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(X_train)
    y_pred = kde_clf.score_samples(X_test)
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred <= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)
    
print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")

print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'KDE', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


Energy thleshold to detect anomaly : -185.816
Energy thleshold to detect anomaly : -194.050
Energy thleshold to detect anomaly : -184.582
Energy thleshold to detect anomaly : -68.318
Energy thleshold to detect anomaly : -183.603
 Precision = 0.527 ± 0.033
 Recall    = 0.526 ± 0.030
 F1-Score  = 0.527 ± 0.032
 F1-scores:
           0
0  0.518467
1  0.518541
2  0.490023
3  0.585871
4  0.520385


## 6. DA-GMM

In [36]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]


    input_size = X_train.shape[-1]
    # Define and fit the dagmm model

    model_dagmm = DAGMM(input_size=input_size, comp_hiddens=[60, 30, 10], comp_activation="selu", 
                  est_hiddens=[15, 3], est_activation="selu", kernel_initializer="glorot_normal", est_dropout_ratio=0.2,
                  n_epochs=20, batch_size=128, learning_rate=0.001, normalize='MinMax', patience=20, random_seed=111)
    with HiddenPrints():
        model_dagmm.fit(X_train, X_test, y_test) 

    y_pred = model_dagmm.predict(X_test)

    # Energy thleshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

# results.append({'models': 'DA-GMM', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


Energy thleshold to detect anomaly : -35.585
Energy thleshold to detect anomaly : -36.250
Energy thleshold to detect anomaly : -34.859
Energy thleshold to detect anomaly : -34.772
Energy thleshold to detect anomaly : -35.315
 Precision = 0.551 ± 0.011
 Recall    = 0.551 ± 0.008
 F1-Score  = 0.551 ± 0.009
 F1-scores:
           0
0  0.538892
1  0.550550
2  0.565549
3  0.555401
4  0.543239


In [39]:

df_results.to_pickle('pd_results_spring_10%.pkl')

## 7. NN-MPPCA

In [14]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]

    input_size = X_train.shape[-1]
    # Define and fit the dagmm model

    nn_mppca = NNMPPCA(input_size=input_size, est_hiddens=[15, 3], est_activation="selu", kernel_initializer="glorot_normal", 
                  est_dropout_ratio=0.2, n_epochs=10, batch_size=1028, lambda1=1, lambda2=0.5, lambda3=0.01,
                  learning_rate=0.001, patience=20, normalize='MinMax', random_seed=42)
    nn_mppca.para_init(X_train)

    with HiddenPrints():
        nn_mppca.fit(X_train, X_test, y_test)

    y_pred = nn_mppca.predict(X_test) 
    # Energy thleshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'NN-MPPCA', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


Energy thleshold to detect anomaly : 50.656
Energy thleshold to detect anomaly : 50.814
Energy thleshold to detect anomaly : 50.987
Energy thleshold to detect anomaly : 50.953
Energy thleshold to detect anomaly : 50.933
 Precision = 0.487 ± 0.005
 Recall    = 0.491 ± 0.003
 F1-Score  = 0.489 ± 0.004
 F1-scores:
           0
0  0.497129
1  0.487649
2  0.488326
3  0.485840
4  0.486643


## VAE

In [15]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    input_size = X_train.shape[-1]

    # Define and fit the damppca model

    vae = VAE(input_size=input_size, hidden_sizes=[60, 30, 10], activation="selu", kernel_initializer="glorot_normal",
                      n_epochs=200, batch_size=256, learning_rate=0.001, patience=20, normalize='Standard')
    with HiddenPrints():
        vae.fit(X_train, X_test, y_test)

    y_pred = vae.predict(X_test) 
    # Energy thleshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'VAE', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})

Energy thleshold to detect anomaly : -0.595
Energy thleshold to detect anomaly : -0.755
Energy thleshold to detect anomaly : -0.879
Energy thleshold to detect anomaly : -0.857
Energy thleshold to detect anomaly : -0.684
 Precision = 0.521 ± 0.008
 Recall    = 0.522 ± 0.008
 F1-Score  = 0.521 ± 0.008
 F1-scores:
           0
0  0.513904
1  0.522894
2  0.513763
3  0.535242
4  0.521610


## DONUT

In [16]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    input_size = X_train.shape[-1]

    # Define and fit the damppca model

    donut = DONUT(input_size=input_size, hidden_sizes=[60, 30, 10], activation="selu", kernel_initializer="glorot_normal",
                      n_epochs=200, batch_size=256, learning_rate=0.001, patience=20, normalize='Standard')
    with HiddenPrints():
        donut.fit(X_train, X_test, y_test)

    y_pred = donut.predict(X_test) 
    # Energy thleshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'DONUT', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})

Energy thleshold to detect anomaly : -2.869
Energy thleshold to detect anomaly : -8.251
Energy thleshold to detect anomaly : -7.355
Energy thleshold to detect anomaly : -7.982
Energy thleshold to detect anomaly : -7.964
 Precision = 0.629 ± 0.016
 Recall    = 0.631 ± 0.015
 F1-Score  = 0.630 ± 0.015
 F1-scores:
           0
0  0.609664
1  0.623637
2  0.630072
3  0.656625
4  0.628999


## DA-MPPCA

In [17]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    input_size = X_train.shape[-1]

    # Define and fit the damppca model

    damppca = DAMPPCA(input_size=input_size, comp_hiddens=[60, 30, 10], comp_activation="selu", 
                      est_hiddens=[15, 3], est_activation="selu", kernel_initializer="glorot_normal", est_dropout_ratio=0.2,
                      n_epochs=200, batch_size=256, lambda1=0.1, lambda2=0.5, lambda3=0.05, lambda4=0.001, learning_rate=0.001, 
                      patience=20, normalize='Standard', random_seed=42)
    damppca.para_init(X_train)
    with HiddenPrints():

        damppca.fit(X_train, X_test, y_test)

    y_pred = damppca.predict(X_test) 
    # Energy thleshold to detect anomaly = 50% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'DA-MPPCA', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})

Energy thleshold to detect anomaly : -7.913
Energy thleshold to detect anomaly : -6.312
Energy thleshold to detect anomaly : -10.899
Energy thleshold to detect anomaly : -9.171
Energy thleshold to detect anomaly : -10.527
 Precision = 0.545 ± 0.014
 Recall    = 0.543 ± 0.013
 F1-Score  = 0.544 ± 0.013
 F1-scores:
           0
0  0.569622
1  0.534100
2  0.533030
3  0.539125
4  0.544280


## DV-MPPCA

In [18]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    input_size = X_train.shape[-1]

    # Define and fit the damppca model

    vaemppca = VAEMPPCA(input_size=input_size, comp_hiddens=[60, 30, 10], comp_activation="selu", 
                      est_hiddens=[15, 3], est_activation="selu", kernel_initializer="glorot_normal", est_dropout_ratio=0.2,
                      n_epochs=200, batch_size=256, lambda1=1, lambda2=1, lambda3=1, lambda4=0.001, learning_rate=0.001, 
                      patience=20, normalize='Standard')
    vaemppca.para_init(X_train)
    with HiddenPrints():
        vaemppca.fit(X_train, X_test, y_test)

    y_pred = vaemppca.predict(X_test) 
    # Energy threshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy threshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'DV-MPPCA', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})


Energy threshold to detect anomaly : 0.068
Energy threshold to detect anomaly : 0.022
Energy threshold to detect anomaly : 0.018
Energy threshold to detect anomaly : 0.003
Energy threshold to detect anomaly : 0.013
 Precision = 0.642 ± 0.013
 Recall    = 0.642 ± 0.011
 F1-Score  = 0.642 ± 0.012
 F1-scores:
           0
0  0.630824
1  0.624926
2  0.652717
3  0.647444
4  0.653953


## DV-MPPCA1

In [108]:
precs = []
recals = []
fscores = []

for i in range(6):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    input_size = X_train.shape[-1]

    # Define and fit the damppca model

    vaemppca = VAEMPPCA1(input_size=input_size, comp_hiddens=[60, 30, 10], comp_activation="selu", 
                      est_hiddens=[15, 3], est_activation="selu", kernel_initializer="glorot_normal", est_dropout_ratio=0.2,
                      n_epochs=200, batch_size=128, lambda1=1, lambda2=1, lambda3=1, lambda4=0.001, learning_rate=0.001, 
                      patience=10, normalize='Standard')
    vaemppca.para_init(X_train)
    with HiddenPrints():
        vaemppca.fit(X_train, X_test, y_test)

    y_pred = vaemppca.predict(X_test) 
    # Energy threshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy threshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

# results.append({'models': 'DV-MPPCA1', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})

Energy threshold to detect anomaly : -5.625
Energy threshold to detect anomaly : -7.149
Energy threshold to detect anomaly : -5.036
Energy threshold to detect anomaly : -6.491
Energy threshold to detect anomaly : -6.370
Energy threshold to detect anomaly : -5.265
 Precision = 0.630 ± 0.027
 Recall    = 0.633 ± 0.027
 F1-Score  = 0.632 ± 0.027
 F1-scores:
           0
0  0.632952
1  0.575522
2  0.646142
3  0.628837
4  0.655203
5  0.650500


## DV-MPPCA2

In [20]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    input_size = X_train.shape[-1]

    # Define and fit the damppca model

    vaemppca = VAEMPPCA2(input_size=input_size, comp_hiddens=[60, 30, 10], comp_activation="selu", 
                      est_hiddens=[15, 3], est_activation="selu", kernel_initializer="glorot_normal", est_dropout_ratio=0.2,
                      n_epochs=200, batch_size=256, lambda1=1, lambda2=1, lambda3=1, lambda4=0.001, learning_rate=0.001, 
                      patience=20, normalize='Standard')
    vaemppca.para_init(X_train)
    with HiddenPrints():
        vaemppca.fit(X_train, X_test, y_test)

    y_pred = vaemppca.predict(X_test) 
    # Energy threshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy threshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

results.append({'models': 'DV-MPPCA2', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})

Energy threshold to detect anomaly : -4.101
Energy threshold to detect anomaly : -2.182
Energy threshold to detect anomaly : -2.108
Energy threshold to detect anomaly : -2.214
Energy threshold to detect anomaly : -3.616
 Precision = 0.668 ± 0.015
 Recall    = 0.669 ± 0.012
 F1-Score  = 0.668 ± 0.014
 F1-scores:
           0
0  0.659091
1  0.671492
2  0.675245
3  0.688164
4  0.648002


## DV-MPPCA3

In [108]:
precs = []
recals = []
fscores = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_injected, y_bi, test_size=0.50)
    X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
    input_size = X_train.shape[-1]

    # Define and fit the damppca model

    vaemppca = VAEMPPCA3(input_size=input_size, comp_hiddens=[60, 30, 10], comp_activation="selu", 
                      est_hiddens=[15, 3], est_activation="selu", kernel_initializer="glorot_normal", est_dropout_ratio=0.2,
                      n_epochs=200, batch_size=128, lambda1=1, lambda2=1, lambda3=1, lambda4=0.001, learning_rate=0.001, 
                      patience=10, normalize='Standard')
    vaemppca.para_init(X_train)
    with HiddenPrints():
        vaemppca.fit(X_train, X_test, y_test)

    y_pred = vaemppca.predict(X_test) 
    # Energy threshold to detect anomaly = 80% percentile of energies
    anomaly_energy_threshold = np.percentile(y_pred, 50)
    print(f"Energy threshold to detect anomaly : {anomaly_energy_threshold:.3f}")
    # Detect anomalies from test data
    y_pred_flag = np.where(y_pred >= anomaly_energy_threshold, 1, 0)
    prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
    precs.append(prec)
    
    recals.append(recall)
    fscores.append(fscore)

print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")
print(f" F1-scores:\n", pd.DataFrame(fscores))

# results.append({'models': 'DV-MPPCA1', 'Precision': precs, 'Recall': recals, 'Fscores': fscores})

Energy threshold to detect anomaly : 1.822
Energy threshold to detect anomaly : 0.598
Energy threshold to detect anomaly : 17.938
Energy threshold to detect anomaly : 16.359
Energy threshold to detect anomaly : 2.134
 Precision = 0.547 ± 0.029
 Recall    = 0.545 ± 0.025
 F1-Score  = 0.546 ± 0.027
 F1-scores:
           0
0  0.584312
1  0.561062
2  0.503427
3  0.539424
4  0.543082


In [51]:
recals.remove(recals[2])

In [109]:
df_results = load_file('pd_results_spring_50%.pkl')
df_results

,models,Precision,Recall,Fscores
0,GMM,"[0.523015873015873, 0.5263888888888889, 0.5359...","[0.5259377494014366, 0.5281704160860043, 0.530...","[0.5244727417429367, 0.5272781476696811, 0.533..."
1,MPPCA,"[0.5503968253968254, 0.5386904761904762, 0.536...","[0.5441349548842683, 0.5389043271139341, 0.538...","[0.5472479779049122, 0.5387973804326256, 0.537..."
2,IF,"[0.48115079365079366, 0.4876984126984127, 0.48...","[0.48685003011443484, 0.48054740957966763, 0.4...","[0.4839836343678276, 0.48409650418513045, 0.48..."
3,OC,"[0.5801587301587302, 0.5708333333333333, 0.564...","[0.5792393026941363, 0.576437587657784, 0.5651...","[0.5796986518636003, 0.5736217725052336, 0.564..."
4,KDE,"[0.5194444444444445, 0.5160714285714286, 0.489...","[0.5174935758054952, 0.5210336538461539, 0.490...","[0.5184671749678186, 0.5185406698564594, 0.490..."
5,DA-GMM,"[0.5226190476190476, 0.515079365079365, 0.5204...","[0.524074810982889, 0.5158982511923689, 0.5205...","[0.5233459169481424, 0.5154884829229548, 0.520..."
6,NN-MPPCA,"[0.4982142857142857, 0.4857142857142857, 0.485...","[0.49604899249308576, 0.4896, 0.49116820553994...","[0.49712928133042966, 0.48764940239043825, 0.4..."
7,VAE,"[0.5115079365079365, 0.5234126984126984, 0.512...","[0.5163228519927899, 0.5223762376237624, 0.514...","[0.5139041164158278, 0.5228939544103073, 0.513..."
8,DONUT,"[0.6083333333333333, 0.6240079365079365, 0.628...","[0.6110003985651654, 0.6232659532302814, 0.631...","[0.6096639490952476, 0.6236367241721197, 0.630..."
9,DA-MPPCA,"[0.5718253968253968, 0.5345238095238095, 0.534...","[0.567434534357157, 0.533676703645008, 0.53134...","[0.5696215041011958, 0.5340999206978588, 0.533..."


In [58]:
df_results.Fscores.apply(np.mean)

0     0.725944
1     0.844172
2     0.653423
3     0.827374
4     0.577658
5     0.097851
6     0.834813
7     0.883528
8     0.789542
9     0.591727
10    0.918382
11    0.934962
12    0.871602
Name: Fscores, dtype: float64

In [110]:
df_results.loc[13] = 'DV-MPPCA3', precs, recals, fscores
df_results

,models,Precision,Recall,Fscores
0,GMM,"[0.523015873015873, 0.5263888888888889, 0.5359...","[0.5259377494014366, 0.5281704160860043, 0.530...","[0.5244727417429367, 0.5272781476696811, 0.533..."
1,MPPCA,"[0.5503968253968254, 0.5386904761904762, 0.536...","[0.5441349548842683, 0.5389043271139341, 0.538...","[0.5472479779049122, 0.5387973804326256, 0.537..."
2,IF,"[0.48115079365079366, 0.4876984126984127, 0.48...","[0.48685003011443484, 0.48054740957966763, 0.4...","[0.4839836343678276, 0.48409650418513045, 0.48..."
3,OC,"[0.5801587301587302, 0.5708333333333333, 0.564...","[0.5792393026941363, 0.576437587657784, 0.5651...","[0.5796986518636003, 0.5736217725052336, 0.564..."
4,KDE,"[0.5194444444444445, 0.5160714285714286, 0.489...","[0.5174935758054952, 0.5210336538461539, 0.490...","[0.5184671749678186, 0.5185406698564594, 0.490..."
5,DA-GMM,"[0.5226190476190476, 0.515079365079365, 0.5204...","[0.524074810982889, 0.5158982511923689, 0.5205...","[0.5233459169481424, 0.5154884829229548, 0.520..."
6,NN-MPPCA,"[0.4982142857142857, 0.4857142857142857, 0.485...","[0.49604899249308576, 0.4896, 0.49116820553994...","[0.49712928133042966, 0.48764940239043825, 0.4..."
7,VAE,"[0.5115079365079365, 0.5234126984126984, 0.512...","[0.5163228519927899, 0.5223762376237624, 0.514...","[0.5139041164158278, 0.5228939544103073, 0.513..."
8,DONUT,"[0.6083333333333333, 0.6240079365079365, 0.628...","[0.6110003985651654, 0.6232659532302814, 0.631...","[0.6096639490952476, 0.6236367241721197, 0.630..."
9,DA-MPPCA,"[0.5718253968253968, 0.5345238095238095, 0.534...","[0.567434534357157, 0.533676703645008, 0.53134...","[0.5696215041011958, 0.5340999206978588, 0.533..."


In [111]:
df_results.to_pickle('pd_results_spring_50%.pkl')

In [23]:
# pd_results.drop(index=15, inplace=True)
# # pd_results.loc[6.5] = 'VAE', precs, recals, fscores
# pd_results.sort_index(inplace=True)
# pd_results.reset_index(drop=True, inplace=True)
# pd_results

# 5. Local Outlier Factor (LOF)

In [24]:
# from sklearn.neighbors import LocalOutlierFactor
# precs = []
# recals = []
# fscores = []
# for i in range(10):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)
#     X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
# #     X_train_reg, y_train_reg = X_train[y_train == 0], y_train[y_train == 0]
# #     X_train_out, y_train_out = X_train[y_train == 1], y_train[y_train == 1]
# #     X_train_out_sub, _, y_train_out_sub, _ = train_test_split(X_train_out, y_train_out, test_size=1)
# #     X_train = np.concatenate((X_train_reg, X_train_out_sub), axis=0)
# #     y_train = np.concatenate((y_train_reg, y_train_out_sub), axis=0)

        
# #     scaler = StandardScaler()
# #     X_train = scaler.fit_transform(X_train)
# #     X_test = scaler.transform(X_test)

#     lof_clf = LocalOutlierFactor(n_neighbors=20, contamination=0.001, novelty=True).fit(X_train)
#     y_pred = lof_clf.score_samples(X_test)
#     anomaly_energy_threshold = np.percentile(y_pred, 20)
#     print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
#     # Detect anomalies from test data
#     y_pred_flag = np.where(y_pred <= anomaly_energy_threshold, 1, 0)
    
# #     y_pred = el_clf.predict(X_test)
# #     y_pred_flag = np.where(y_pred==-1, 1, 0)
    
#     prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
#     precs.append(prec)
#     recals.append(recall)
#     fscores.append(fscore)
    
# print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
# print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
# print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")

# 6. Covariance determinant estimator

In [25]:
# from sklearn.covariance import EllipticEnvelope
# precs = []
# recals = []
# fscores = []
# for i in range(10):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)
#     X_train, y_train = X_train[y_train == 0], y_train[y_train == 0]
# #     X_train_reg, y_train_reg = X_train[y_train == 0], y_train[y_train == 0]
# #     X_train_out, y_train_out = X_train[y_train == 1], y_train[y_train == 1]
# #     X_train_out_sub, _, y_train_out_sub, _ = train_test_split(X_train_out, y_train_out, test_size=1)
# #     X_train = np.concatenate((X_train_reg, X_train_out_sub), axis=0)
# #     y_train = np.concatenate((y_train_reg, y_train_out_sub), axis=0)

        
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)

#     el_clf = EllipticEnvelope(assume_centered=True, random_state=0).fit(X_train)
# #     y_pred = el_clf.score_samples(X_test)
# #     anomaly_energy_threshold = np.percentile(y_pred, 20)
# #     print(f"Energy thleshold to detect anomaly : {anomaly_energy_threshold:.3f}")
# #     # Detect anomalies from test data
# #     y_pred_flag = np.where(y_pred <= anomaly_energy_threshold, 1, 0)
#     y_pred = el_clf.predict(X_test)
#     y_pred_flag = np.where(y_pred==-1, 1, 0)
#     prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred_flag, average="binary")
#     precs.append(prec)
#     recals.append(recall)
#     fscores.append(fscore)
    
# print(f" Precision = {np.mean(precs):.3f} \u00B1 {np.std(precs):.3f}")
# print(f" Recall    = {np.mean(recals):.3f} \u00B1 {np.std(recals):.3f}")
# print(f" F1-Score  = {np.mean(fscores):.3f} \u00B1 {np.std(fscores):.3f}")